In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 12: Hill Climbing Algorithm ---</h2><p>You try contacting the Elves using your <span title="When you look up the specs for your handheld device, every field just says &quot;plot&quot;.">handheld device</span>, but the river you're following must be too low to get a decent signal.</p>
<p>You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where <code>a</code> is the lowest elevation, <code>b</code> is the next-lowest, and so on up to the highest elevation, <code>z</code>.</p>
<p>Also included on the heightmap are marks for your current position (<code>S</code>) and the location that should get the best signal (<code>E</code>). Your current position (<code>S</code>) has elevation <code>a</code>, and the location that should get the best signal (<code>E</code>) has elevation <code>z</code>.</p>
<p>You'd like to reach <code>E</code>, but to save energy, you should do it in <em>as few steps as possible</em>. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be <em>at most one higher</em> than the elevation of your current square; that is, if your current elevation is <code>m</code>, you could step to elevation <code>n</code>, but not to elevation <code>o</code>. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)</p>
<p>For example:</p>
<pre><code><em>S</em>abqponm
abcryxxl
accsz<em>E</em>xk
acctuvwj
abdefghi
</code></pre>
<p>Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the <code>e</code> at the bottom. From there, you can spiral around to the goal:</p>
<pre><code>v..v&lt;&lt;&lt;&lt;
&gt;v.vv&lt;&lt;^
.&gt;vv&gt;E^^
..v&gt;&gt;&gt;^^
..&gt;&gt;&gt;&gt;&gt;^
</code></pre>
<p>In the above diagram, the symbols indicate whether the path exits each square moving up (<code>^</code>), down (<code>v</code>), left (<code>&lt;</code>), or right (<code>&gt;</code>). The location that should get the best signal is still <code>E</code>, and <code>.</code> marks unvisited squares.</p>
<p>This path reaches the goal in <code><em>31</em></code> steps, the fewest possible.</p>
<p><em>What is the fewest steps required to move from your current position to the location that should get the best signal?</em></p>
</article>


In [2]:
from collections import deque


tests = [
    {
        "name": "Example",
        "s": """
            Sabqponm
            abcryxxl
            accszExk
            acctuvwj
            abdefghi
        """,
        "expected": 31,
    },
]


class Grid:
    def __init__(self, s: str) -> None:
        grid = [list(l.strip()) for l in s.strip().splitlines()]
        rows, cols = len(grid), len(grid[0])

        self.grid = [[-1] * cols for _ in range(rows)]

        for row, col in product(range(rows), range(cols)):
            if grid[row][col] == "S":
                self.S = row, col
                grid[row][col] = "a"
            elif grid[row][col] == "E":
                self.E = row, col
                grid[row][col] = "z"

            self.grid[row][col] = ord(grid[row][col])

    def fewest_steps(self) -> int:
        rows, cols = len(self.grid), len(self.grid[0])

        queue = deque([(0, self.S)])
        seen = set()

        while queue:
            steps, current = queue.pop()

            if current in seen:
                continue

            seen.add(current)

            if current == self.E:
                return steps

            r, c = current
            for dr, dc in ((-1, 0), (0, 1), (1, 0), (0, -1)):
                if (
                    0 <= r + dr < rows
                    and 0 <= c + dc < cols
                    and self.grid[r + dr][c + dc] - self.grid[r][c] <= 1
                ):
                    queue.appendleft((steps + 1, (r + dr, c + dc)))

    def __str__(self):
        return "\n".join("".join(chr(h) for h in row) for row in self.grid)


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Grid(s).fewest_steps()


Test Example passed, for test_part_I.
Success


In [3]:
with open("../input/day12.txt") as f:
    puzzle = f.read()

print(f"Part I: {Grid(puzzle).fewest_steps()}")

Part I: 440


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>440</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.</p>
<p>To maximize exercise while hiking, the trail should start as low as possible: elevation <code>a</code>. The goal is still the square marked <code>E</code>. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from <em>any square at elevation <code>a</code></em> to the square marked <code>E</code>.</p>
<p>Again consider the example from above:</p>
<pre><code><em>S</em>abqponm
abcryxxl
accsz<em>E</em>xk
acctuvwj
abdefghi
</code></pre>
<p>Now, there are six choices for starting position (five marked <code>a</code>, plus the square marked <code>S</code> that counts as being at elevation <code>a</code>). If you start at the bottom-left square, you can reach the goal most quickly:</p>
<pre><code>...v&lt;&lt;&lt;&lt;
...vv&lt;&lt;^
...v&gt;E^^
.&gt;v&gt;&gt;&gt;^^
&gt;^&gt;&gt;&gt;&gt;&gt;^
</code></pre>
<p>This path reaches the goal in only <code><em>29</em></code> steps, the fewest possible.</p>
<p><em>What is the fewest steps required to move starting from any square with elevation <code>a</code> to the location that should get the best signal?</em></p>
</article>


In [4]:
tests = [
    {
        "name": "Example",
        "s": """
            Sabqponm
            abcryxxl
            accszExk
            acctuvwj
            abdefghi
        """,
        "expected": 29,
    },
]


class Grid:
    def __init__(self, s: str) -> None:
        grid = [list(l.strip()) for l in s.strip().splitlines()]
        rows, cols = len(grid), len(grid[0])

        self.grid = [[-1] * cols for _ in range(rows)]
        self.S = []

        for row, col in product(range(rows), range(cols)):
            if grid[row][col] == "S":
                grid[row][col] = "a"
            elif grid[row][col] == "E":
                self.E = row, col
                grid[row][col] = "z"

            if grid[row][col] == "a":
                self.S.append((row, col))
            self.grid[row][col] = ord(grid[row][col])

    def fewest_steps(self) -> int:
        rows, cols = len(self.grid), len(self.grid[0])

        queue = deque((0, s) for s in self.S)
        seen = set()

        while queue:
            steps, current = queue.popleft()

            if current in seen:
                continue

            seen.add(current)

            if current == self.E:
                return steps

            r, c = current
            for dr, dc in ((-1, 0), (0, 1), (1, 0), (0, -1)):
                if (
                    0 <= r + dr < rows
                    and 0 <= c + dc < cols
                    and self.grid[r + dr][c + dc] - self.grid[r][c] <= 1
                ):
                    queue.append((steps + 1, (r + dr, c + dc)))

    def __str__(self):
        return "\n".join("".join(chr(h) for h in row) for row in self.grid)


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Grid(s).fewest_steps()


Test Example passed, for test_part_I.
Success


In [5]:
print(f"Part II: { Grid(puzzle).fewest_steps()}")

Part II: 439


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>439</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
